# Analyzing Used Car Listings on eBay Kleinanzeigen
The aim of this project is to clean the dataset of used cars from eBay Kleinanzigen
and analyze the included used car listings.<br>
The dataset was originally scraped and uploaded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data). The version of the dataset we are working with is a sample of 50,000 data points that was prepared by [Dataquest](https://www.dataquest.io/) including simulating a less-cleaned version of the data. <br><br>
The data dictionary provided with data is as follows:<br>

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which which year the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which which year the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeenOnline - When the crawler saw this ad last online.


In [0]:
#Import the pandas and NumPy libraries
import pandas as pd
import numpy as np

In [0]:
#Read the autos.csv CSV file into pandas
autos=pd.read_csv("autos.csv",encoding="Latin-1")

In [0]:
#print information about the autos dataframe, as well as the first few rows.
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


-Notice that the columns names are not consistent: some of the labels contain a mixture of capital letters, lowercase letters and whitespace.The column names use camelcase instead of Python's preferred snakecase. <br> Therefore, the first step will be cleaning the columns labels. <br>
-Also notice that the "name" column is very lengthy and can be splitted into smaller more meaningful segments. <br>
-Also notice that some columns contain null values, yet none has more than ~20% null values. <br>

# Column names cleaning

-Convert camelcase to snakecase <br>
-Reword some columns labels to more meaningful names <br>

In [0]:
#print the columns' labels
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [0]:
autos.columns=['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [0]:
#look at descriptive statistics for all columns 
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 17:37:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Notice the following: <br>
- the "seller", "offer type", "ab_test","gearbox" and "unrepaired_damage" colunms contain each only two values, making them candidates to be dropped. <br>
- The "num_photos" have no unique values and all the values are zeros, making it another candidate to be dropped.
- The "registration_year" and "power_ps" columns need to be investigated further becuase they seem to have abnormal maximum values. <br>
- The "price" and "odometer: columns are numeric values stored as text. <br>

# Cleaning "price" and "odometer" columns
- Remove any non-numeric characters. <br>
- Convert the column to a numeric dtype. <br>
- Rename the column to odometer_km. <br>

In [0]:
autos["odometer"]=(autos["odometer"].str.replace("km","")
                                     .str.replace(",","")
                                     .astype(int))
autos["odometer"].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer, dtype: int64

In [0]:
autos.rename(columns={"odometer":"odometer_km"},inplace=True)
autos["odometer_km"].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int64

In [0]:
autos["price"]=(autos["price"].str.replace("$","")
                                     .str.replace(",","")
                                     .astype(int))
autos["price"].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

## Further Analysis of the "odometer_km" and "price" column 

We will:
- Analyze the columns using minimum and maximum values 
- Look for any values that look unrealistically high or low (outliers) that we might want to remove.

In [0]:
#first check how many unique values are in the "price" column
autos["price"].unique().shape[0]

2357

In [0]:
#and in the "odometer_km" column as well
autos["odometer_km"].unique().shape[0]

13

In [0]:
#now check for the counts of each unique value in the "price" column in ascending order
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

In [0]:
#now check for the counts of each unique value in the "price" column in descending order
autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

- We notice that there are unrealistic car prices that we need to investigate. The prices increases steadily from 0 till 350000, then after 350000 the price jump to 999990. <br>
- We will remove the entries with prices higher than 350000 or prices equal to 0

In [0]:
autos=autos[autos["price"].between(1,350001)]
autos["price"].value_counts().sort_index(ascending=False).head(20)

350000    1
345000    1
299000    1
295000    1
265000    1
259000    1
250000    1
220000    1
198000    1
197000    1
194000    1
190000    1
180000    1
175000    1
169999    1
169000    1
163991    1
163500    1
155000    1
151990    1
Name: price, dtype: int64

In [0]:
#now check for the counts of each unique value in the "odometer_km" column in ascending order
autos["odometer_km"].value_counts().sort_index(ascending=False).head(20)

150000    31414
125000     5057
100000     2115
90000      1734
80000      1415
70000      1217
60000      1155
50000      1012
40000       815
30000       780
20000       762
10000       253
5000        836
Name: odometer_km, dtype: int64

# Exploring the date columns

In [0]:
#first we need to understand how the values in the three string columns are formatted.
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


We note that these columns all represent full timestamp values

Now, we will calculate the distribution of values in the "date_crawled", "ad_created" and "last_seen" columns as percentages

In [0]:
#we start by the "date_crawled" column
(autos["date_crawled"].str[:10]
                      .value_counts(normalize=True,dropna=False)
                      .sort_index()
                      *100)

2016-03-05    2.532688
2016-03-06    1.404304
2016-03-07    3.601359
2016-03-08    3.329558
2016-03-09    3.308967
2016-03-10    3.218367
2016-03-11    3.257490
2016-03-12    3.691959
2016-03-13    1.566972
2016-03-14    3.654896
2016-03-15    3.428395
2016-03-16    2.960980
2016-03-17    3.162772
2016-03-18    1.291053
2016-03-19    3.477813
2016-03-20    3.788737
2016-03-21    3.737259
2016-03-22    3.298672
2016-03-23    3.222485
2016-03-24    2.934212
2016-03-25    3.160712
2016-03-26    3.220426
2016-03-27    3.109235
2016-03-28    3.486050
2016-03-29    3.409863
2016-03-30    3.368681
2016-03-31    3.183363
2016-04-01    3.368681
2016-04-02    3.547823
2016-04-03    3.860805
2016-04-04    3.648718
2016-04-05    1.309585
2016-04-06    0.317101
2016-04-07    0.140019
Name: date_crawled, dtype: float64

In [0]:
(autos["date_crawled"].str[:10]
                      .value_counts(normalize=True,dropna=False)
                      .sort_values()
                      *100)

2016-04-07    0.140019
2016-04-06    0.317101
2016-03-18    1.291053
2016-04-05    1.309585
2016-03-06    1.404304
2016-03-13    1.566972
2016-03-05    2.532688
2016-03-24    2.934212
2016-03-16    2.960980
2016-03-27    3.109235
2016-03-25    3.160712
2016-03-17    3.162772
2016-03-31    3.183363
2016-03-10    3.218367
2016-03-26    3.220426
2016-03-23    3.222485
2016-03-11    3.257490
2016-03-22    3.298672
2016-03-09    3.308967
2016-03-08    3.329558
2016-03-30    3.368681
2016-04-01    3.368681
2016-03-29    3.409863
2016-03-15    3.428395
2016-03-19    3.477813
2016-03-28    3.486050
2016-04-02    3.547823
2016-03-07    3.601359
2016-04-04    3.648718
2016-03-14    3.654896
2016-03-12    3.691959
2016-03-21    3.737259
2016-03-20    3.788737
2016-04-03    3.860805
Name: date_crawled, dtype: float64

We note that the site was crawled daily over a one month period in March and April 2016. 

In [0]:
#Then the "last_seen" column
(autos["last_seen"].str[:10]
                      .value_counts(normalize=True,dropna=False)
                      .sort_index()
                      *100)

2016-03-05     0.107073
2016-03-06     0.432410
2016-03-07     0.539483
2016-03-08     0.741275
2016-03-09     0.959539
2016-03-10     1.066612
2016-03-11     1.237517
2016-03-12     2.378256
2016-03-13     0.889529
2016-03-14     1.260167
2016-03-15     1.587563
2016-03-16     1.645218
2016-03-17     2.808607
2016-03-18     0.735097
2016-03-19     1.583445
2016-03-20     2.065273
2016-03-21     2.063214
2016-03-22     2.137342
2016-03-23     1.853186
2016-03-24     1.976732
2016-03-25     1.921137
2016-03-26     1.680222
2016-03-27     1.564913
2016-03-28     2.085864
2016-03-29     2.234119
2016-03-30     2.477093
2016-03-31     2.378256
2016-04-01     2.279419
2016-04-02     2.491506
2016-04-03     2.520334
2016-04-04     2.448265
2016-04-05    12.476063
2016-04-06    22.180583
2016-04-07    13.194688
Name: last_seen, dtype: float64

We note that in the last three days of the crawling period, the last_seen percentages witness a spike, which might be an indication that there had been a huge volume of sales happening in these last three days. However, it is more likely that it is indication of the ending of the crawling period rather than an increase in the sales.

In [0]:
#Then the "ad_created" column
(autos["ad_created"].str[:10]
                      .value_counts(normalize=True,dropna=False)
                      .sort_index()
                      *100)

2015-06-11    0.002059
2015-08-10    0.002059
2015-09-09    0.002059
2015-11-10    0.002059
2015-12-05    0.002059
2015-12-30    0.002059
2016-01-03    0.002059
2016-01-07    0.002059
2016-01-10    0.004118
2016-01-13    0.002059
2016-01-14    0.002059
2016-01-16    0.002059
2016-01-22    0.002059
2016-01-27    0.006177
2016-01-29    0.002059
2016-02-01    0.002059
2016-02-02    0.004118
2016-02-05    0.004118
2016-02-07    0.002059
2016-02-08    0.002059
2016-02-09    0.002059
2016-02-11    0.002059
2016-02-12    0.004118
2016-02-14    0.004118
2016-02-16    0.002059
2016-02-17    0.002059
2016-02-18    0.004118
2016-02-19    0.006177
2016-02-20    0.004118
2016-02-21    0.006177
                ...   
2016-03-09    3.315145
2016-03-10    3.189540
2016-03-11    3.290435
2016-03-12    3.675486
2016-03-13    1.700813
2016-03-14    3.518995
2016-03-15    3.401627
2016-03-16    3.012458
2016-03-17    3.127767
2016-03-18    1.359003
2016-03-19    3.368681
2016-03-20    3.794914
2016-03-21 

In [0]:
(autos["ad_created"].str[:10]
                      .value_counts(normalize=True,dropna=False)
                      .sort_values()
                      *100)

2016-02-09    0.002059
2016-02-22    0.002059
2016-02-07    0.002059
2015-06-11    0.002059
2016-01-03    0.002059
2016-01-14    0.002059
2016-02-11    0.002059
2016-01-13    0.002059
2016-01-07    0.002059
2016-01-29    0.002059
2015-09-09    0.002059
2016-01-22    0.002059
2015-08-10    0.002059
2016-02-01    0.002059
2016-02-16    0.002059
2015-11-10    0.002059
2016-01-16    0.002059
2015-12-05    0.002059
2016-02-17    0.002059
2016-02-08    0.002059
2015-12-30    0.002059
2016-02-05    0.004118
2016-02-02    0.004118
2016-02-24    0.004118
2016-02-26    0.004118
2016-02-14    0.004118
2016-02-20    0.004118
2016-02-18    0.004118
2016-02-12    0.004118
2016-01-10    0.004118
                ...   
2016-03-06    1.531967
2016-03-13    1.700813
2016-03-05    2.289715
2016-03-24    2.928035
2016-03-16    3.012458
2016-03-27    3.098940
2016-03-17    3.127767
2016-03-25    3.175126
2016-03-31    3.187481
2016-03-10    3.189540
2016-03-23    3.206013
2016-03-26    3.226604
2016-03-22 

We note that most of the cars' ads were created in the crawling period.
There is a large variety of ad created dates, and oldest dates back to June 2015. 

# Exploring the registration year column

In [0]:
#let's check the distribution of the "registration_year" column
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

We note that the maximum (9999) and minimum (1000) registration years are unrealistic, therefore we need to further investigate these anomalies.

In [0]:
autos["registration_year"].value_counts().sort_index()

1000       1
1001       1
1111       1
1800       2
1910       5
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       3
1951       2
1952       1
1953       1
1954       2
1955       2
1956       4
1957       2
1958       4
1959       6
1960      23
1961       6
1962       4
1963       8
1964      12
        ... 
2000    3156
2001    2636
2002    2486
2003    2699
2004    2703
2005    2936
2006    2670
2007    2277
2008    2215
2009    2085
2010    1589
2011    1623
2012    1310
2013     803
2014     663
2015     392
2016    1220
2017    1392
2018     470
2019       2
2800       1
4100       1
4500       1
4800       1
5000       4
5911       1
6200       1
8888       1
9000       1
9999       3
Name: registration_year, Length: 95, dtype: int64

- Since a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.
- There it is safe to only count the cars with registration years falling in the interval 1900-2016.

In [0]:
#Remove the values outside the interval and calculate the distribution of the remaining values
autos=autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).sort_values()*100

1952    0.002142
1953    0.002142
1943    0.002142
1929    0.002142
1931    0.002142
1938    0.002142
1948    0.002142
1927    0.002142
1939    0.002142
1955    0.004284
1957    0.004284
1934    0.004284
1951    0.004284
1941    0.004284
1954    0.004284
1950    0.006427
1962    0.008569
1937    0.008569
1958    0.008569
1956    0.008569
1910    0.010711
1959    0.012853
1961    0.012853
1963    0.017138
1964    0.025706
1965    0.036417
1975    0.038560
1969    0.040702
1976    0.044986
1977    0.047128
          ...   
1985    0.203509
1988    0.289197
1989    0.372743
1991    0.726206
1990    0.743343
1992    0.792614
2015    0.839742
1993    0.910435
1994    1.347443
2014    1.420278
2013    1.720186
2016    2.613483
1995    2.628478
2012    2.806281
1996    2.941239
2010    3.403954
2011    3.476789
1997    4.179431
2009    4.466485
2008    4.744971
2007    4.877788
1998    5.062017
2002    5.325507
2001    5.646837
2006    5.719672
2003    5.781796
2004    5.790364
1999    6.2059

We note that most of the cars were registered in the last 20 years

## Converting dates to numeric data

We will convert the dates to be uniform numeric data, <br>
for example, "2016-03-21" becomes the integer 20160321.

In [0]:
autos["ad_created"]=autos["ad_created"].str.replace("-","")
autos["ad_created"].head()

0    20160326 00:00:00
1    20160404 00:00:00
2    20160326 00:00:00
3    20160312 00:00:00
4    20160401 00:00:00
Name: ad_created, dtype: object

In [0]:
autos["last_seen"]=autos["last_seen"].str.replace("-","")
autos["last_seen"].head()

0    20160406 06:45:54
1    20160406 14:45:08
2    20160406 20:15:37
3    20160315 03:16:28
4    20160401 14:38:50
Name: last_seen, dtype: object

In [0]:
autos["date_crawled"]=autos["date_crawled"].str.replace("-","")
autos["date_crawled"].head()

0    20160326 17:47:46
1    20160404 13:38:56
2    20160326 18:57:24
3    20160312 16:58:10
4    20160401 14:38:50
Name: date_crawled, dtype: object

# Exploring price by brand

In [0]:
#first we will check which car brands we will be investigating
autos["brand"].value_counts(normalize=True).sort_values(ascending=False)*100

volkswagen        21.126368
bmw               11.004477
opel              10.758124
mercedes_benz      9.646323
audi               8.656627
ford               6.989996
renault            4.714980
peugeot            2.984083
fiat               2.564212
seat               1.827296
skoda              1.640925
nissan             1.527388
mazda              1.518819
smart              1.415994
citroen            1.400998
toyota             1.270324
hyundai            1.002549
sonstige_autos     0.981127
volvo              0.914719
mini               0.876159
mitsubishi         0.822604
honda              0.784045
kia                0.706926
alfa_romeo         0.664082
porsche            0.612669
suzuki             0.593389
chevrolet          0.569825
chrysler           0.351321
dacia              0.263490
daihatsu           0.250637
jeep               0.227073
subaru             0.214220
land_rover         0.209936
saab               0.164949
jaguar             0.156381
daewoo             0

We will find the mean price for each of the top 20% car brands in the dataset

In [0]:
#create a subset of the top 20% of the car brands
selected_brands=(autos["brand"]
                 .value_counts()
                 .sort_values(ascending=False)
                 .head(
                     int(
                     autos["brand"]
                         .unique()
                         .shape[0]*0.2)
                 ))
selected_brands

volkswagen       9862
bmw              5137
opel             5022
mercedes_benz    4503
audi             4041
ford             3263
renault          2201
peugeot          1393
Name: brand, dtype: int64

Now we will:
- Create an empty dictionary to hold the aggregate data.
- Loop over the selected brands, and assign the mean price to the dictionary, with the brand name as the key.

In [0]:
mean_price_dict={}
for brand in selected_brands.index:
  mean_price_dict[brand]=autos.loc[autos["brand"]==brand,"price"].mean()
sorted(mean_price_dict.items(),key =lambda k: k[1])

[('renault', 2474.8646069968195),
 ('opel', 2975.2419354838707),
 ('peugeot', 3094.0172290021537),
 ('ford', 3749.4695065890287),
 ('volkswagen', 5402.410261610221),
 ('bmw', 8332.820517811953),
 ('mercedes_benz', 8628.450366422385),
 ('audi', 9336.687453600594)]

We notice that among the top 20% car brands, there's a distinct price gap. The brand with the lowest average price is renault (~2475), and the brand with the highest average price is audi (~9336).   

# Relating the price, brand and mileage

For the top 20% brands, we will use aggregation to understand the average mileage for those cars and if there's any visible link with mean price.

In [0]:
mean_mileage_dict={}
for brand in selected_brands.index:
  mean_mileage_dict[brand]=autos.loc[autos["brand"]==brand,"odometer_km"].mean()
sorted(mean_mileage_dict.items(),key =lambda k: k[1])

[('ford', 124266.01287159056),
 ('peugeot', 127153.62526920316),
 ('renault', 128071.33121308497),
 ('volkswagen', 128707.15879132022),
 ('audi', 129157.38678544914),
 ('opel', 129310.0358422939),
 ('mercedes_benz', 130788.36331334666),
 ('bmw', 132572.51313996495)]

Let's create a new dataframe to store the top 20% car brands, their mean prices and their mean mileage

In [0]:
#create two series for the mean prices and the mean mileages
mean_prices=pd.Series(mean_price_dict)
mean_mileage=pd.Series(mean_mileage_dict)

In [0]:
#Now create the dataframe using the two series
top_brands=pd.DataFrame(mean_prices,columns=["mean_prices"])
top_brands["mean_mileage"]=mean_mileage
top_brands

,mean_prices,mean_mileage
audi,9336.687454,129157.386785
bmw,8332.820518,132572.513140
ford,3749.469507,124266.012872
mercedes_benz,8628.450366,130788.363313
opel,2975.241935,129310.035842
peugeot,3094.017229,127153.625269
renault,2474.864607,128071.331213
volkswagen,5402.410262,128707.158791


We will check the standard deviation between the top 20% brands' mileages

- We note that for all the top 20% brands, the mean mileages are very close to each other. The range of car mileages does not vary as much as the prices do by brand. 
- There is a slight trend to the more expensive cars to have higher mileage, and the less expensive cars to have lower mileage.

# Investigating the num_photos column

In [0]:
#we will take a closer look
autos["num_photos"].value_counts()

0    46681
Name: num_photos, dtype: int64

We note that the num_photos column have only one value which is 0. Therefore, we can safely remove this column.

In [0]:
autos.drop("num_photos",axis=1,inplace=True)
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'postal_code', 'last_seen'],
      dtype='object')